In [1]:
# %matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt

import os
import shutil
import random

from fashionutils import to_file_path

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.optimizers import Adam

from keras import optimizers

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks \
    import TensorBoard, ModelCheckpoint, \
    Callback, CSVLogger, ReduceLROnPlateau, \
    EarlyStopping

from keras.datasets import cifar10
from keras.utils import np_utils

from keras.preprocessing import image
import numpy as np


Using TensorFlow backend.


In [2]:
IN_DIR = 'images-299/items'

In [3]:
model = load_model('model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0         
__________

In [4]:
model.load_weights('param.hdf5')

In [5]:


model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [6]:
classes = ['accessory','bag','fashion-accessories','hat','jacket-outerwear',
'leg-wear','onepiece','pants','shoes','skirt','tops','wrist-watch']

In [7]:
files = os.listdir(IN_DIR)

In [11]:
os.makedirs('result_images', exist_ok=True)

for count in range(0, 1):

    w = 2
    h = 2

    item_start = w*h*count + 100


    fig, axs = plt.subplots(w, h, figsize=(6*h,6*w))

    for x in range(0, int(w)):
        for y in range(0, int(h)):
            item = item_start + y*w+x

            img = image.load_img(os.path.join(IN_DIR, files[item]), target_size=(299, 299))
            iimg = image.img_to_array(img)
            iimg = np.expand_dims(iimg, axis=0)

            iimg = iimg/255.0

            pred = model.predict(iimg)[0]

            top = 5
            top_indices = pred.argsort()[-top:][::-1]
            result = [(classes[i], pred[i]) for i in top_indices]

            txt = ''
            for r in result:
                txt = txt + str(r) +'\n'

            p = axs[x, y]
            p.axis('off')

            p.imshow(img)
            p.annotate(txt, xy=(20, 280), size=16, color='white')
            p.annotate(txt, xy=(22, 282), size=16, color='black')

    # plt.show()
    plt.savefig(os.path.join('result_images','result_%d.png'%count))